In [6]:
import pandas as pd
import numpy as np
from sqlalchemy import MetaData, create_engine, Table, Column, ForeignKey
from sqlalchemy import Integer, String, Date, Float, Time
from sqlalchemy import exc

In [7]:
#Constantes
DB = "tp3"
HOST = "localhost"
PORT = "5432"
PASS = "postgres"
USER = "postgres"

# Diagrama de DW  - Inmobiliaria

![Diagrama](img/Inmobiliaria.png "Diagrama")

In [8]:
url_conect = USER + ':' + PASS + '@' + HOST+ ':' + PORT + '/'+ DB
engine = create_engine('postgresql://' + url_conect)

In [27]:
metadata = MetaData()


estate = Table('estate', metadata,
              Column('id', Integer, primary_key=True),
              Column('category', String),
              Column('area', Float, nullable=False),
              Column('city', String, nullable=False),
              Column('province', String, nullable=False),
              Column('rooms', Integer, nullable=False),
              Column('bedrooms', Integer, nullable=False),
              Column('garage', Integer),
              Column('meters', Float, nullable=False))

customer = Table('customer', metadata,
                Column('id', Integer, primary_key=True),
                Column('name', String, nullable=False),
                Column('surname', String, nullable=False),
                Column('budget', Float),
                Column('address', String),
                Column('city', String),
                Column('phone', String))

visit = Table('visit', metadata, 
             Column('id', Integer, primary_key=True),
             Column('duration', Integer, nullable=False))

sale = Table('sale', metadata, 
            Column('id', Integer, primary_key=True),
            Column('agreed_price', Float, nullable=False),
            Column('status', String, nullable=False))

fact = Table('fact', metadata,
            Column('id', Integer, primary_key=True),
            Column('id_estate', Integer, ForeignKey('estate.id')),
            Column('id_cust', Integer, ForeignKey('customer.id')),
            Column('date', Date, nullable=False),
            Column('id_visit', Integer, ForeignKey('visit.id'), nullable=True),
            Column('id_sale', Integer, ForeignKey('sale.id'), nullable=True))


In [28]:
metadata.create_all(engine)

Carga de Datos
--------------

In [21]:
try:
    df_estate = pd.read_csv('dataTp3/Inmobilaria/Estate.csv')
    df_estate.to_sql('estate', engine, index=None, if_exists="append")
    
except exc.IntegrityError:
    print("Los datos de Estate ya han sido cargados en la BD")

Los datos de Estate ya han sido cargados en la BD


In [22]:
try:
    df_customer = pd.read_csv('dataTp3/Inmobilaria/Customer.csv')
    df_customer.to_sql('customer', engine, index=None, if_exists="append")
except exc.IntegrityError:
    print("Los datos de Customer ya han sido cargados en la BD")  

Los datos de Customer ya han sido cargados en la BD


In [23]:
try:
    df_visit = pd.read_csv('dataTp3/Inmobilaria/Visit.csv')
    df_visit.to_sql('visit', engine, index=None, if_exists="append")
except exc.IntegrityError:
    print("Los datos de Visit ya han sido cargados en la BD")

Los datos de Visit ya han sido cargados en la BD


In [24]:
try:
    df_sale = pd.read_csv('dataTp3/Inmobilaria/Sale.csv')
    df_sale.to_sql('sale', engine, index=None, if_exists="append")
except exc.IntegrityError:
    print("Los datos de Sale ya han sido cargados en la BD")

Los datos de Sale ya han sido cargados en la BD


In [25]:
try:
    df_fact = pd.read_csv('dataTp3/Inmobilaria/Fact.csv')
    df_fact.to_sql('fact', engine, index=None, if_exists="append")
except exc.IntegrityError:
    print("Los datos de Fact ya han sido cargados en la BD")

# Consultas

## Consulta 1: 
¿Qué tipo de propiedad se vendió por el precio más alto con respecto a cada
ciudad y meses?

In [ ]:
no funciono

## Consulta 2:
¿Quén ha comprado un piso con el precio más alto con respecto a cada mes?

In [27]:
string_sql = """select * from (
                    select mes, 'name', surname, agreed_price, rank() over particion_por_mes
                        from ( select date_part('month',"date") as mes, "name", surname, category, agreed_price
                            from fact inner join estate on (id_estate = estate.id)
                            inner join sale on (id_sale = sale.id) 
                            inner join customer on (id_cust = customer.id)
                        where id_sale is not NULL and category = 'piso'
                        order by agreed_price DESC) as pepe
                        WINDOW particion_por_mes AS (partition by mes order by agreed_price DESC)
                ) as sub_query
                WHERE rank = 1; """

df_clientes_pisos = pd.read_sql_query(string_sql,engine.connect()) 
df_clientes_pisos


,mes,?column?,surname,agreed_price,rank
0,1.0,name,Cook,8116995.0,1
1,2.0,name,Dunn,6368893.0,1
2,3.0,name,Mitchell,9766483.0,1
3,4.0,name,Price,5946760.0,1
4,5.0,name,Elliott,9911554.0,1
5,6.0,name,Thomas,5854180.0,1
6,7.0,name,Payne,6114295.0,1
7,8.0,name,Rose,6010489.0,1
8,10.0,name,George,7998435.0,1
9,11.0,name,Murray,4502727.0,1


## Consulta 3:
¿Cuál es la duración media de visitas en las propiedades de cada categoría?

In [28]:
string_sql = """ select distinct category, cat
                 from (
                        select category, avg(duration)
                        over particion_por_categoria as cat
                        from (
                                fact inner join estate on (id_estate = estate.id) 
                                inner join visit on (id_visit = visit.id) )
                        WINDOW particion_por_categoria AS (partition by category)
                 ) as promedio; """
df_mayor_duracion = pd.read_sql_query(string_sql,engine.connect()) 
df_mayor_duracion


,category,cat
0,departamento,5.319231
1,casa,4.750000
2,duplex,6.375000
3,monoambiente,4.917355
4,triplex,5.250000
5,piso,5.000000
